Ok so lets gooo fire emoji

In [3]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

label_map = {
    "Normal": 0,
    "Depression": 1,
    "Suicidal": 2,
    "Anxiety": 3,
    "Stress": 4,
    "Bipolar": 5,
    "Personality Disorder": 6
}


db = pd.read_csv('Sentiment_analysis_dataset(in).csv')


c:\Users\Laure\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print("Unique labels in dataset:", db['Status'].unique())


Unique labels in dataset: ['Depression' 'Stress' 'Normal' 'Suicidal' 'Anxiety'
 'Personality disorder' 'Bipolar' nan]


In [ ]:
db = db.dropna(subset=['Statement', 'Status'])
db['Status'] = db['Status'].str.strip().str.title()
db['Statement'] = db['Statement'].apply(lambda x: x.lower())
unmapped = set(db['Status'].unique()) - set(label_map.keys())
if unmapped:
    print("Warning: These labels aren't in your map:", unmapped)
    db = db[db['Status'].isin(label_map.keys())]

statement = db['Statement'].tolist()
status = db['Status'].tolist()

print(db.head(20))

                                            Statement      Status
0   life doesn’t feel worth it that’s kind of it? ...  Depression
1   this life sucks and if it were for my religiou...  Depression
2   its been 9 months now for our marriage and she...      Stress
3   i do not feel particularly sad or anxious or a...  Depression
4   i am taking venlafaxine. it is an ssri. does n...  Depression
5   i (17m) have come to the realisation that my d...  Depression
6   been on fluoxetine for year but anyway there s...  Depression
7   even a four day week seems too long i want to ...      Normal
8   i often had suicidal thoughts, recently, with ...    Suicidal
9   this is the first time facing pisces. extro pa...      Normal
10  being an orphan at your early stages of life b...  Depression
11  the dread of thinking you have cancer... again...     Anxiety
12  i (30f) married my husband (31m) about 6 month...      Stress
13           what day is eid al-fitr if you may know?      Normal
14  free c

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(db['Statement'], db['Status'], test_size=0.2, random_state=42)

train_labels = train_labels.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)

print(f"training data size: {len(train_texts)}")
print(f"Testing data size: {len(val_texts)}")

training data size: 29500
Testing data size: 7375


In [7]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True,padding=True)

print(type(train_texts))
print(type(train_texts[0]))


<class 'pandas.core.series.Series'>
<class 'str'>


In [8]:
print(train_texts.head())



36248    i want everyone to feel my pain i am sick of e...
26254    home. had uni today, got results back :hd, yeh...
8331                                          really? why?
26739    i am not sure how much longer i can last. i al...
18584    i feel so trapped in my mind, the feeling that...
Name: Statement, dtype: object


In [9]:
class MentalHealthDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, label_map):
        self.encodings = encodings
        self.labels = labels
        self.label_map = label_map

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        label = self.labels.iloc[idx]
        #item["labels"] = torch.tensor(self.label_map[idx])
        item["labels"] = torch.tensor(self.label_map[label])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MentalHealthDataset(train_encodings, train_labels, label_map)
val_dataset = MentalHealthDataset(val_encodings, val_labels, label_map)

In [10]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 7)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.500100,0.499798
2,0.365400,0.494076
3,0.261800,0.612386


TrainOutput(global_step=11064, training_loss=0.4172975892013859, metrics={'train_runtime': 38499.343, 'train_samples_per_second': 2.299, 'train_steps_per_second': 0.287, 'total_flos': 1.1724410128896e+16, 'train_loss': 0.4172975892013859, 'epoch': 3.0})

The machine restarted a total of 3 times for one reaosn or another, and hence is not complete as of right now

In [12]:
model_path = "./model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('./model\\tokenizer_config.json',
 './model\\special_tokens_map.json',
 './model\\vocab.txt',
 './model\\added_tokens.json')

In [13]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_labels, pred_labels)
print("Accuracy:", accuracy)

NameError: name 'true_labels' is not defined

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_map.keys(), yticklabels=label_map.keys())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()